In [ ]:
#!nvidia-smi

In [1]:
!pip install ultralytics motrackers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
from motrackers import CentroidTracker
from motrackers.utils import draw_tracks
import numpy as np
from ultralytics import YOLO
import time
from google.colab.patches import cv2 #Google Collab tidak bisa pakai cv2 yang original
from google.colab.patches import cv2_imshow
#import cv2

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
main_path = "/content/gdrive/MyDrive/Chicken Health and Behavior Detection"
destination_path = '/content'

In [5]:
!cp "$main_path/models/object_detection/model_yolov11s.zip" "$destination_path"

In [6]:
!unzip -q "$destination_path/model_yolov11s.zip" -d "$destination_path/model"

In [7]:
model = YOLO(f'{destination_path}/model/yolov11s.pt')

In [8]:
video_path = f'{main_path}/data/processed/video_test.mp4'
output_path = f'{destination_path}/output_video_test.avi'

In [9]:
# Open video
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Video writer
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))

# Tracker
tracker = CentroidTracker(max_lost=5, tracker_output_format='mot_challenge')
frame_count = 0

In [10]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    start_time = time.time()

    # Run YOLOv11 inference
    results = model(frame)

    # Get predictions
    detections = []
    colors = []

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()       # Bounding box coordinates
        confidences = result.boxes.conf.cpu().numpy() # Confidence scores
        class_ids = result.boxes.cls.cpu().numpy()    # Class IDs

        for i, box in enumerate(boxes):
            x1, y1, x2, y2 = box.astype(int)
            w = x2 - x1
            h = y2 - y1
            detections.append([x1, y1, w, h])
            colors.append((0, 255, 0))

    # Update tracker
    tracks = tracker.update(np.array(detections), confidences, class_ids)

    # Draw tracks
    tracked_frame = draw_tracks(frame, tracks)

    # Show Frame Slices
    #if frame_count % 10 == 0:
    #    cv2_imshow(tracked_frame)
    #    if cv2.waitKey(1) & 0xFF == ord('q'):
    #        break

    # Write to output video
    out.write(tracked_frame)
    frame_count += 1
    print(f"Processed frame {frame_count} | Detection + Tracking Time: {time.time() - start_time:.3f}s")


0: 384x640 62 0s, 129.3ms
Speed: 19.8ms preprocess, 129.3ms inference, 445.5ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 1 | Detection + Tracking Time: 7.096s

0: 384x640 61 0s, 15.9ms
Speed: 2.8ms preprocess, 15.9ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 2 | Detection + Tracking Time: 0.061s

0: 384x640 61 0s, 21.3ms
Speed: 2.7ms preprocess, 21.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 3 | Detection + Tracking Time: 0.055s

0: 384x640 60 0s, 12.8ms
Speed: 2.6ms preprocess, 12.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 4 | Detection + Tracking Time: 0.040s

0: 384x640 60 0s, 12.6ms
Speed: 2.3ms preprocess, 12.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 5 | Detection + Tracking Time: 0.036s

0: 384x640 60 0s, 12.6ms
Speed: 3.0ms preprocess, 12.6ms inference, 2.1ms postprocess per image at shape (1, 

In [12]:
cap.release()
out.release()